### Main Libraries Needed

In [1]:
#Import of main libraries

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_csv
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

#Import of the pyomo module
from pyomo.environ import *

### Importing and creating necessary Data Frames

In [14]:
#Import Bids, Lanes, and Carrier DF's
bidsDf = read_csv("Nordics Bids.csv")
bidsDf = bidsDf.drop("Unnamed: 0", axis = 1)
lanesDf = read_csv("Nordics Lanes.csv")
lanesDf = lanesDf.drop("Unnamed: 0", axis = 1)
CarriersDf = read_csv("Carriers.csv")
CarriersDf = CarriersDf.drop("Unnamed: 0", axis = 1)

In [16]:
#Create data frames for delta matrix
deltaDf = DataFrame(np.zeros((len(lanesDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    deltaDf.at[bidsDf.loc[bid,'Cluster Lot Index'] - 1, bid] = 1

In [18]:
# Create the eta DF
etaDf = DataFrame(np.zeros((len(CarriersDf.index), len(bidsDf.index))))
for bid in bidsDf.index:
    etaDf.at[bidsDf.loc[bid,'Carrier Index'] - 1, bid] = 1

In [19]:
import pandas as pd

In [20]:
Mc = pd.Series(np.sum(etaDf, axis = 1))

In [25]:
Mc

0      0.0
1     13.0
2     14.0
3     17.0
4      5.0
5      0.0
6      0.0
7      2.0
8      9.0
9      8.0
10     0.0
11     8.0
12    17.0
13     2.0
14    10.0
15     6.0
dtype: float64

### Create the necessary dictionaries

In [21]:
carriers = Mc.to_dict()

bidValues = dict()
for bid in bidsDf.index:
    bidValues[bid] = bidsDf.loc[bid, 'Cost']

demandValues = dict()
for lane in lanesDf.index:
    demandValues[lane] = 1

delta = dict()
for lane in lanesDf.index:
    for bid in bidsDf.index:
        delta[(lane,bid)] = deltaDf.loc[lane,bid]

eta = dict()
for c in CarriersDf.index:
    for bid in bidsDf.index:
        eta[(c,bid)] = etaDf.loc[c,bid]


### Model Parameters

In [26]:
# Create the Concrete Model
model = ConcreteModel()

model.numBids = len(bidsDf.index)

model.numItems = len(lanesDf.index)

model.numCarriers = len(CarriersDf.index)

model.BIDS = Set(initialize = bidsDf.index.values)

model.LANES = Set(initialize = lanesDf.index.values)

model.CARRIERS = Set( initialize = CarriersDf.index.values)

model.M = Param(model.CARRIERS, initialize = carriers)

model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

# Below is where I made the suggested work around from https://stackoverflow.com/questions/45616967/pyomo-valueerror-invalid-constraint-expression

model.delta = Param(model.LANES, model.BIDS, initialize= delta, doc='delta gives information regarding which lanes are in a bid package')#,mutable = True)

model.eta = Param(model.CARRIERS, model.BIDS, initialize= eta, doc='eta gives information regarding which carriers are in a bid package', mutable = True)

model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

model.z = Var(model.CARRIERS, domain = Binary, doc='Decision variable for each carrier in the program')

model.maxCarriers = 5
model.minCarriers = 1

### Set Model Obj and Constraints

In [27]:
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [28]:
def constraint_rule(model, l):
    return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= model.demand[l]
model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

def constraint2_rule(model, k):
    return sum(model.eta[k,b]*model.x[b] for b in model.BIDS) - model.M[k]*model.z[k]<=0
model.upperBoundConstraint = Constraint(model.CARRIERS, rule=constraint2_rule)

def constraint3_rule(model, k):
    return model.z[k] - sum(model.eta[k,b]*model.x[b] for b in model.BIDS)<=0
model.lowerBoundConstraint = Constraint(model.CARRIERS, rule=constraint3_rule)

def constraint4_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)<=model.maxCarriers
model.zConstraint = Constraint(rule=constraint4_rule)

def constraint5_rule(model):
    return sum(model.z[i] for i in model.CARRIERS)>=model.minCarriers
model.zConstraint2 = Constraint(rule=constraint5_rule)

### Run the Model

In [29]:
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [30]:
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

102 ms ± 4.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 80716.8
  Upper bound: 80716.8
  Number of objectives: 1
  Number of constraints: 53
  Number of variables: 128
  Number of nonzeros: 505
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.029449462890625
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    OBJ:
      Value: 80716.8
  Variable:
    x[0]:
      Value: 1
    x[103]:
      Value: 1
    x[106]:
      Value: 1
    x[10]:
      Value: 1
    x[20]:
      Value: 1
    x[29]:
      Value: 1
    x[37]:
      Value: 1
    x[45]:
      Value: 1
    x[56]:
      Value: 1
    x[65]:
      Value: 1
    x[74]:
      Value: 1
    x[81]:
      Value: 1
    x[85]:
      Value: 1
    x[89]:
      Value: 1
    x[93]:
      Value: 1
    x[96]:
      Value: 1
   

In [32]:
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 80716.8
  Upper bound: 80716.8
  Number of objectives: 1
  Number of constraints: 53
  Number of variables: 128
  Number of nonzeros: 505
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.029449462890625
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

In [0]:
#model.pprint()

In [33]:
winningBids = []
index = 0
bidNum = 0
for bids in range(len(bidsDf.index)):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

winningBidsDf = bidsDf.iloc[winningBids]
winningBidsDf.head()

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,DNHG@ISO_181,DNHG,ISO_181,181,3127.2,1,3127.2,0.0,1,Nordics,3,1
10,SUTI@ISO_182,SUTI,ISO_182,182,3920.4,1,3920.4,0.0,12,Nordics,14,2
20,RMCL@ISO_183,RMCL,ISO_183,183,2400.0,1,2400.0,0.0,1,Nordics,12,3
29,RMCL@ISO_184,RMCL,ISO_184,184,4194.0,1,4194.0,0.0,3,Nordics,12,4
37,RMCL@ISO_185,RMCL,ISO_185,185,4194.0,1,4194.0,0.0,3,Nordics,12,5


In [34]:
winningBidsDf

,Bid,Carrier,Lot,Lot#,Cost,Rank,Lowest Bid,Delta vs Lowest,Shipments per Lot,Site Cluster,Carrier Index,Cluster Lot Index
0,DNHG@ISO_181,DNHG,ISO_181,181,3127.2,1,3127.2,0.0,1,Nordics,3,1
10,SUTI@ISO_182,SUTI,ISO_182,182,3920.4,1,3920.4,0.0,12,Nordics,14,2
20,RMCL@ISO_183,RMCL,ISO_183,183,2400.0,1,2400.0,0.0,1,Nordics,12,3
29,RMCL@ISO_184,RMCL,ISO_184,184,4194.0,1,4194.0,0.0,3,Nordics,12,4
37,RMCL@ISO_185,RMCL,ISO_185,185,4194.0,1,4194.0,0.0,3,Nordics,12,5
45,SUTI@ISO_186,SUTI,ISO_186,186,2942.4,1,2942.4,0.0,26,Nordics,14,6
56,RMCL@ISO_187,RMCL,ISO_187,187,2670.0,1,2670.0,0.0,1,Nordics,12,7
65,VTGT@ISO_188,VTGT,ISO_188,188,2400.0,1,2400.0,0.0,7,Nordics,16,8
74,RMCL@ISO_189,RMCL,ISO_189,189,4560.0,1,4560.0,0.0,1,Nordics,12,9
81,STTC@ISO_218,STTC,ISO_218,218,6835.2,1,6835.2,0.0,6,Nordics,13,10


In [0]:
Total = sum(winningBidsDf['Cost']*winningBidsDf['Shipments per Lot'])
Total

In [35]:
winningBidsDf.to_csv('Nordics Carrier Constraint WinningBids.csv')